In [ ]:
!unzip -q /content/drive/MyDrive/FML_data.zip

In [3]:
!pip install ordered_set

In [4]:
import json
# read the CSV file
import pandas as pd
import numpy as np

import glob
import os

import torch
import torch.utils.data as data_utils
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

# from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from random import shuffle
from ordered_set import OrderedSet

import warnings
warnings.filterwarnings('ignore')


In [5]:
# file names
json_fname = "/content/t20s_json/1001349.json"
csv_fname = "/content/t20s_csv/1001349.csv"

# Opening JSON file
f = open(json_fname)

# returns JSON object as a dictionary
data = json.load(f)

In [6]:
# team details
team_details = {}

In [7]:
teams = []
# get the teams playing in this CSV/JSON id.
for key in data["info"].keys():
    if key == "teams":
        teams = data["info"]["teams"]

print(teams)

['Australia', 'Sri Lanka']


In [8]:
# this cell is for model building persepective
# and not for the dataset analysis purpose.

if teams[0] in team_details.keys():
    # data present atleast of some players.
    for player in data["info"]["players"][teams[0]]:
        # if the player name is not present, means he is a debutant [ASSUMED]
        # then we will average the scores of all others in the team
        if not (player in team_details[teams[0]]["Name"].unique()):
            avg_economy = team_details[teams[0]][:, "Bowling_Economy"].mean()
            avg_avg = team_details[teams[0]][:, "Bowling_Average"].mean()
            avg_sr = team_details[teams[0]][:, "Bowling_Strike_Rate"].mean()

            new_data = {"Name": player,
                        "Bowling_Economy": avg_economy,
                        "Bowling_Average": avg_avg,
                        "Bowling_Strike_Rate": avg_sr}
            team_details[teams[0]] = team_details[teams[0]].append(new_data, ignore_index=True)

else:
    # new team which has never been encountered in the training set as of now.
    df = pd.DataFrame(columns = ['Name', 'Bowling_Economy', 'Bowling_Average', 'Bowling_Strike_Rate'])
    team_details[teams[0]] = df

In [9]:
# get the player names from the JSON File.
for key in data["info"].keys():
    if key == "players":
        print(data["info"]["players"][teams[0]])
        print(data["info"]["players"][teams[1]])

['AJ Finch', 'M Klinger', 'TM Head', 'MC Henriques', 'AJ Turner', 'JP Faulkner', 'TD Paine', 'PJ Cummins', 'A Zampa', 'B Stanlake', 'AJ Tye']
['N Dickwella', 'WU Tharanga', 'EMDY Munaweera', 'DAS Gunaratne', 'TAM Siriwardana', 'CK Kapugedera', 'S Prasanna', 'JRMVB Sanjaya', 'SL Malinga', 'KMDN Kulasekara', 'PADLR Sandakan']


In [10]:
df = pd.read_csv(csv_fname)
df.head()

,Team,Over,Batter,Bowler,Non_Striker,Batter_Runs,Extras,Total_Runs
0,Australia,0,AJ Finch,SL Malinga,M Klinger,0,0,0
1,Australia,0,AJ Finch,SL Malinga,M Klinger,0,0,0
2,Australia,0,AJ Finch,SL Malinga,M Klinger,1,0,1
3,Australia,0,M Klinger,SL Malinga,AJ Finch,2,0,2
4,Australia,0,M Klinger,SL Malinga,AJ Finch,0,0,0


In [11]:
team1 = [x for _, x in df.groupby(df['Team'] == teams[0])]
team1[0].head()

,Team,Over,Batter,Bowler,Non_Striker,Batter_Runs,Extras,Total_Runs
125,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,1,0,1
126,Sri Lanka,0,WU Tharanga,PJ Cummins,N Dickwella,0,1,1
127,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,0,0,0
128,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,0,0,0
129,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,3,0,3


In [12]:
team1[0] = team1[0].reset_index(drop=True)
team1[0].head()

,Team,Over,Batter,Bowler,Non_Striker,Batter_Runs,Extras,Total_Runs
0,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,1,0,1
1,Sri Lanka,0,WU Tharanga,PJ Cummins,N Dickwella,0,1,1
2,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,0,0,0
3,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,0,0,0
4,Sri Lanka,0,N Dickwella,PJ Cummins,WU Tharanga,3,0,3


In [13]:
team1[1].head()

,Team,Over,Batter,Bowler,Non_Striker,Batter_Runs,Extras,Total_Runs
0,Australia,0,AJ Finch,SL Malinga,M Klinger,0,0,0
1,Australia,0,AJ Finch,SL Malinga,M Klinger,0,0,0
2,Australia,0,AJ Finch,SL Malinga,M Klinger,1,0,1
3,Australia,0,M Klinger,SL Malinga,AJ Finch,2,0,2
4,Australia,0,M Klinger,SL Malinga,AJ Finch,0,0,0


In [14]:
# gets the overs bowled information by individual bowlers
bowls_bowled = team1[1].groupby(["Bowler"])["Over"].count()
overs_bowled = bowls_bowled // 6
print(overs_bowled)
print(bowls_bowled)

Bowler
DAS Gunaratne      1
JRMVB Sanjaya      3
KMDN Kulasekara    4
PADLR Sandakan     4
S Prasanna         4
SL Malinga         4
Name: Over, dtype: int64
Bowler
DAS Gunaratne       6
JRMVB Sanjaya      19
KMDN Kulasekara    26
PADLR Sandakan     24
S Prasanna         25
SL Malinga         25
Name: Over, dtype: int64


In [15]:
# gets the runs conceded information by individual bowlers
runs_conceded = team1[1].groupby(["Bowler"])["Total_Runs"].sum()
runs_conceded

Bowler
DAS Gunaratne      11
JRMVB Sanjaya      35
KMDN Kulasekara    38
PADLR Sandakan     31
S Prasanna         24
SL Malinga         29
Name: Total_Runs, dtype: int64

In [16]:
def wicket_condition(prev, curr):
    if prev[0] == curr[0] and prev[1] == curr[1]:
        return 0
    if prev[0] == curr[1] and prev[1] == curr[0]:
        return 0
    return 1

In [17]:
wicket = []

for idx, row in team1[1].iterrows():
    if idx > 0:
        prev_batters = [team1[1].iloc[idx - 1]["Batter"], team1[1].iloc[idx - 1]["Non_Striker"]]
        curr_batters = [team1[1].iloc[idx]["Batter"], team1[1].iloc[idx]["Non_Striker"]]

        if wicket_condition(prev_batters, curr_batters):
            wicket.append(1)
        else:
            wicket.append(0)


# if the wicket falls in the last ball, it is not caputred by the logic
# hence, we assume that no wicket ever falls in the last ball of the innings.
wicket.append(0)

In [18]:
team1[1]["Wickets"] = wicket

team1[1].tail()

,Team,Over,Batter,Bowler,Non_Striker,Batter_Runs,Extras,Total_Runs,Wickets
120,Australia,19,JP Faulkner,KMDN Kulasekara,TD Paine,2,0,2,0
121,Australia,19,JP Faulkner,KMDN Kulasekara,TD Paine,0,1,1,0
122,Australia,19,JP Faulkner,KMDN Kulasekara,TD Paine,1,0,1,1
123,Australia,19,JP Faulkner,KMDN Kulasekara,PJ Cummins,2,0,2,0
124,Australia,19,JP Faulkner,KMDN Kulasekara,PJ Cummins,4,0,4,0


In [19]:
# wickets taken by the bowler
wickets_taken = team1[1].groupby(["Bowler"])["Wickets"].sum()
wickets_taken

Bowler
DAS Gunaratne      1
JRMVB Sanjaya      1
KMDN Kulasekara    1
PADLR Sandakan     1
S Prasanna         0
SL Malinga         2
Name: Wickets, dtype: int64

In [20]:
# average of the bowler
# runs conceded by bowler per wicket taken

average_in_the_match = runs_conceded / wickets_taken
average_in_the_match

Bowler
DAS Gunaratne      11.0
JRMVB Sanjaya      35.0
KMDN Kulasekara    38.0
PADLR Sandakan     31.0
S Prasanna          inf
SL Malinga         14.5
dtype: float64

In [21]:
# strike rate of bowler
# bowls bowled by bowler per wicket taken

strike_rate_in_match = bowls_bowled / wickets_taken
strike_rate_in_match

Bowler
DAS Gunaratne       6.0
JRMVB Sanjaya      19.0
KMDN Kulasekara    26.0
PADLR Sandakan     24.0
S Prasanna          inf
SL Malinga         12.5
dtype: float64

In [22]:
# economy of bowler
# runs conceded per over

economy = runs_conceded / overs_bowled
economy

Bowler
DAS Gunaratne      11.000000
JRMVB Sanjaya      11.666667
KMDN Kulasekara     9.500000
PADLR Sandakan      7.750000
S Prasanna          6.000000
SL Malinga          7.250000
dtype: float64

In [23]:

bowler_details = pd.DataFrame(columns=['Name', 'Bowls_bowled', 'Wickets_taken', 'Runs_conceded'])
csv_path = "/content/t20s_csv"
json_path = "/content/t20s_json"

num_matches = {}

In [24]:
for files in sorted(os.listdir(csv_path)):
    fname = files.split('.')[0]
    json_name = json_path + '/' + fname + '.json'

    # Opening JSON file
    f = open(json_name)
    # returns JSON object as a dictionary
    data = json.load(f)
    # teams
    teams = data["info"]["teams"]

    if teams[0] in num_matches.keys():
        num_matches[teams[0]] += 1
    else:
        num_matches[teams[0]] = 1

    if teams[1] in num_matches.keys():
        num_matches[teams[1]] += 1
    else:
        num_matches[teams[1]] = 1


    # initialize the dataframe
    for team in teams:
        for player in data["info"]["players"][team]:
            if player not in bowler_details['Name'].values:
                bowler_details.loc[len(bowler_details)] = [player, 0, 0, 0]


    f.close()



In [25]:
# num_matches is a dictionary for count of matches played by each country
len(num_matches.keys())

101

In [26]:
def wicket_condition(prev, curr):
    if prev[0] == curr[0] and prev[1] == curr[1]:
        return 0
    if prev[0] == curr[1] and prev[1] == curr[0]:
        return 0
    return 1

In [27]:
for files in sorted(os.listdir(csv_path)):
    fname = csv_path + '/' + files
    df = pd.read_csv(fname)

    wicket = []

    for idx, row in df.iterrows():
        if idx > 0:
            prev_batters = [df.iloc[idx - 1]["Batter"], df.iloc[idx - 1]["Non_Striker"]]
            curr_batters = [df.iloc[idx]["Batter"], df.iloc[idx]["Non_Striker"]]

            if wicket_condition(prev_batters, curr_batters):
                wicket.append(1)
            else:
                wicket.append(0)

    wicket.append(0)

    df["Wickets"] = wicket

    # gets the runs conceded information by individual bowlers
    runs_conceded = df.groupby(["Bowler"])["Total_Runs"].sum()
    wickets_taken = df.groupby(["Bowler"])["Wickets"].sum()
    bowls_bowled = df.groupby(["Bowler"])["Over"].count()


    for items in runs_conceded.iteritems():
        bowler = items[0]
        runs = int(items[1])
        idx = bowler_details.index[bowler_details['Name'] == bowler].tolist()
        bowler_details['Runs_conceded'].iloc[idx[0]] += runs


    for items in wickets_taken.iteritems():
        bowler = items[0]
        wickets = int(items[1])
        idx = bowler_details.index[bowler_details['Name'] == bowler].tolist()
        bowler_details['Wickets_taken'].iloc[idx[0]] += wickets

    for items in bowls_bowled.iteritems():
        bowler = items[0]
        bowls = int(items[1])
        idx = bowler_details.index[bowler_details['Name'] == bowler].tolist()
        bowler_details['Bowls_bowled'].iloc[idx[0]] += bowls

    # Runs_conceded
    df.head()


In [28]:
bowler_details.shape

(4985, 4)

In [29]:
bowler_details.head(30)

,Name,Bowls_bowled,Wickets_taken,Runs_conceded
0,AJ Finch,12,0,27
1,M Klinger,0,0,0
2,TM Head,37,1,56
3,MC Henriques,142,7,198
4,AJ Turner,83,4,83
5,JP Faulkner,523,38,707
6,TD Paine,0,0,0
7,PJ Cummins,1154,65,1388
8,A Zampa,1590,84,1856
9,B Stanlake,443,29,556


In [30]:
temp = bowler_details.drop(bowler_details[bowler_details["Bowls_bowled"] == 0].index)
temp = temp.reset_index(drop = True)
temp.head()

,Name,Bowls_bowled,Wickets_taken,Runs_conceded
0,AJ Finch,12,0,27
1,TM Head,37,1,56
2,MC Henriques,142,7,198
3,AJ Turner,83,4,83
4,JP Faulkner,523,38,707


In [31]:
temp.shape

(3318, 4)

In [32]:
average = temp["Runs_conceded"] / temp["Wickets_taken"]
average.replace([np.inf, -np.inf], 0, inplace=True)

In [33]:
strike_rate = temp["Bowls_bowled"] / temp["Wickets_taken"]
strike_rate.replace([np.inf, -np.inf], 0, inplace = True)

In [34]:
economy = temp["Runs_conceded"] / (temp["Bowls_bowled"] // 6)
economy.replace([np.inf, -np.inf], 0, inplace = True)

In [35]:
average.describe()

count    3318.000000
mean       20.590262
std        18.083401
min         0.000000
25%        11.168478
50%        18.000000
75%        26.000000
max       184.000000
dtype: float64

In [36]:
strike_rate.describe()

count    3318.000000
mean       17.968919
std        13.812961
min         0.000000
25%        11.750000
50%        16.524731
75%        22.278708
max       147.000000
dtype: float64

In [37]:
economy.describe()

count    3318.000000
mean        7.332380
std         2.704515
min         0.000000
25%         5.750000
50%         7.023894
75%         8.363636
max        27.500000
dtype: float64

In [38]:
temp["Average"] = average
temp["StrikeRate"] = strike_rate
temp["Economy"] = economy
temp.head()

,Name,Bowls_bowled,Wickets_taken,Runs_conceded,Average,StrikeRate,Economy
0,AJ Finch,12,0,27,0.000000,0.000000,13.500000
1,TM Head,37,1,56,56.000000,37.000000,9.333333
2,MC Henriques,142,7,198,28.285714,20.285714,8.608696
3,AJ Turner,83,4,83,20.750000,20.750000,6.384615
4,JP Faulkner,523,38,707,18.605263,13.763158,8.126437


In [39]:
temp.replace([np.inf, -np.inf], 0, inplace = True)
temp.head()

,Name,Bowls_bowled,Wickets_taken,Runs_conceded,Average,StrikeRate,Economy
0,AJ Finch,12,0,27,0.000000,0.000000,13.500000
1,TM Head,37,1,56,56.000000,37.000000,9.333333
2,MC Henriques,142,7,198,28.285714,20.285714,8.608696
3,AJ Turner,83,4,83,20.750000,20.750000,6.384615
4,JP Faulkner,523,38,707,18.605263,13.763158,8.126437


In [40]:
bowler_info = temp
bowler_info.head()

,Name,Bowls_bowled,Wickets_taken,Runs_conceded,Average,StrikeRate,Economy
0,AJ Finch,12,0,27,0.000000,0.000000,13.500000
1,TM Head,37,1,56,56.000000,37.000000,9.333333
2,MC Henriques,142,7,198,28.285714,20.285714,8.608696
3,AJ Turner,83,4,83,20.750000,20.750000,6.384615
4,JP Faulkner,523,38,707,18.605263,13.763158,8.126437


In [41]:
avg_avg = bowler_info.loc[:, 'Average'].mean()
avg_s_rate = bowler_info.loc[:, 'StrikeRate'].mean()
avg_eco = bowler_info.loc[:, 'Economy'].mean()

In [42]:
avg_avg, avg_s_rate, avg_eco

(20.590261831412025, 17.968919057517997, 7.332379674884246)

## Batting Statistics computation

In [43]:

def get_batting_pos_dict(df):
#     print ('called !!')
    match_pos_dict = {}
    pos_count = 1
    ns_list = df['Non_Striker'].unique()
    

    batting_order_set = OrderedSet()
    for i in range(len(df)):
        batting_order_set.add(df['Batter'].iloc[i])
        batting_order_set.add(df['Non_Striker'].iloc[i])

    for index in range(len(batting_order_set)):
        match_pos_dict[batting_order_set[index]] = index + 1

    return match_pos_dict

In [44]:
def create_batting_pos_dict(train_files, root = './'):
    batting_pos_arr_dict = {}

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))
        match_pos_dict = {}
        pos_count = 1
        for i in range(len(df)):
            b = df['Batter'][i]

            if b not in batting_pos_arr_dict.keys():
                batting_pos_arr_dict[b] = np.zeros(12)

            if i!=0 and df.iloc[i-1,0] != df.iloc[i,0]: # innings change
                pos_count = 1

            if b not in match_pos_dict.keys():
                match_pos_dict[b] = pos_count
                pos_count +=1

    #     try:
        for b, pos in match_pos_dict.items():
            batting_pos_arr_dict[b][pos-1] +=1

    batting_pos_dict = {}

    for b in batting_pos_arr_dict.keys():
        batting_pos_dict[b] = np.argmax(batting_pos_arr_dict[b]) +1

    return batting_pos_dict

In [45]:
def create_batter_avg_dict(train_files, root):

    batter_runs_dict = {}
    batter_inns_dict = {}

    batter_set = set()

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))

        batter_list = df['Batter'].unique()
        batter_set = batter_set.union(set(batter_list))

        for b in batter_list:
            df_b = df[df['Batter']==b]
            batter_team = df_b['Team'].iloc[0]
            df_team = df[df['Team']==batter_team]

            is_not_out = (df_team['Batter'].iloc[-1] == b) | (df_team['Non_Striker'].iloc[-1] == b)
            batter_runs = df_b['Batter_Runs'].sum()
            batter_balls = len(df_b)

            if b in batter_runs_dict.keys():
                batter_runs_dict[b] += batter_runs
                if not is_not_out:
                    batter_inns_dict[b] += 1
            else:
                batter_runs_dict[b] = batter_runs
                if not is_not_out:
                    batter_inns_dict[b] = 1
                else:
                    batter_inns_dict[b] = 0

    batters_avg = {}
    for b in batter_set:
        if batter_inns_dict[b] > 0:
            batters_avg[b] = batter_runs_dict[b]/batter_inns_dict[b]
        else:
             batters_avg[b] = batter_runs_dict[b]

    return batters_avg

def create_batter_sr_dict(train_files, root):
    batter_runs_dict = {}
    batter_balls_dict = {}
    batter_set = set()

    for file in train_files:
        df = pd.read_csv(os.path.join(root, file))

        batter_list = df['Batter'].unique()
        batter_set = batter_set.union(set(batter_list))

        for b in batter_list:
            df_b = df[df['Batter']==b]
            batter_runs = df_b['Batter_Runs'].sum()
            batter_balls = len(df_b)

            if b in batter_runs_dict.keys():
                batter_runs_dict[b] += batter_runs
                batter_balls_dict[b] += batter_balls
            else:
                batter_runs_dict[b] = batter_runs
                batter_balls_dict[b] = batter_balls

    batters_sr = {}
    for b in batter_set:
        batters_sr[b] = batter_runs_dict[b]/batter_balls_dict[b]*100

    return batters_sr

def create_batter_stats(train_files, root = './'):

    batter_avg_dict = create_batter_avg_dict(train_files, root)
    batter_sr_dict = create_batter_sr_dict(train_files, root)

    return batter_avg_dict, batter_sr_dict


In [46]:
def create_group_batter_stats(batters_avg, batters_sr, batting_pos_dict):
    grp_avg = np.zeros(3)
    grp_sr = np.zeros(3)
    grp_count = np.zeros(3)

    batters = batting_pos_dict.keys()

    for b in batters:
        if batting_pos_dict[b] <=6:
            grp_avg[0] += batters_avg[b]
            grp_sr[0] += batters_sr[b]
            grp_count[0] +=1
        elif batting_pos_dict[b] >= 7 and batting_pos_dict[b] <= 9:
            grp_avg[1] += batters_avg[b]
            grp_sr[1] += batters_sr[b]
            grp_count[1] +=1
        else:
            grp_avg[2] += batters_avg[b]
            grp_sr[2] += batters_sr[b]
            grp_count[2] +=1

    grp_avg /= grp_count
    grp_sr /= grp_count

    return grp_avg, grp_sr

In [47]:
def get_batter_features(batter, batting_pos, batter_stats, batter_group_stats):
    batter_avg_dict, batter_sr_dict = batter_stats
    grp_avg, grp_sr = batter_group_stats

    if batter in batter_avg_dict.keys():
        return (batter_avg_dict[batter], batter_sr_dict[batter])
    else:
        # get group features
        if batting_pos <=6:
            return grp_avg[0], grp_sr[0]
        elif batting_pos >= 7 and batting_pos <= 9:
            return grp_avg[1], grp_sr[1]
        else:
            return grp_avg[2], grp_sr[2]

In [48]:
all_files = os.listdir('/content/t20s_csv/')
split = int(len(all_files) * 0.8)
train_files = all_files[:split]
test_files = all_files[split:]

In [49]:
train_batting_pos_dict = create_batting_pos_dict(train_files, '/content/t20s_csv/')
train_batter_avg_dict, train_batter_sr_dict = create_batter_stats(train_files, '/content/t20s_csv/')
train_grp_avg, train_grp_sr = create_group_batter_stats(train_batter_avg_dict, train_batter_sr_dict, train_batting_pos_dict)

In [50]:
# provided by Naay
current_over = None
ball_count = 0

# Function to generate dynamic over numbers
def generate_over(row):
    global current_over, ball_count
    if row['Over'] != current_over:
        current_over = row['Over']
        ball_count = 1
    else:
        ball_count += 1
    return f"{current_over}.{ball_count}"


In [51]:
# T20 Data
path = r't20s_csv/'
all_files = glob.glob(path + "/*.csv")
shuffle(all_files)
innings2 = []
target=[]

for filename in all_files:
    json_path = "/content/t20s_json"
    indiv_fname = filename.split('/')[-1][:-4] + '.json'
    json_fname = json_path + '/' + indiv_fname

    df = pd.read_csv(filename)
    f = open(json_fname)
    data = json.load(f)

    # to handle cases like No Result, Rain Stopped Play, Draw etc
    if "winner" in data["info"]["outcome"].keys():
        winteam = data["info"]["outcome"]["winner"]
    else:
        continue



    # first batting team
    f_in_team = df['Team'].iloc[0]
    innings = []
    for idx, val in enumerate(df.iterrows()):
        if df['Team'].iloc[idx] == f_in_team:
            innings.append(1)
        else:
            innings.append(2)

    df["innings"] = innings

    df1 = df[df['innings']==1]
    df1['Team_Runs'] = df1['Total_Runs'].cumsum()
    target_score = df1['Team_Runs'].iloc[-1] +1

    df = df.set_index(df["innings"]).drop(index=1)
    df = df.drop(["innings"], axis = 1)


    b_avgs = []
    b_srts = []
    b_eco = []

    batter_avg = []
    batter_sr = []
    non_striker_avg = []
    non_striker_sr = []


    for idx, val in enumerate(df.iterrows()):
        # updating the bowler score
        # print(df)
        batting_pos_dict = get_batting_pos_dict(df)
        curr_bowler = df['Bowler'].iloc[idx]
        if curr_bowler in bowler_info['Name'].values:
            b_idx = bowler_info.index[bowler_info['Name'] == curr_bowler].tolist()[0]
            avg = bowler_info['Average'].iloc[b_idx]
            srate = bowler_info['StrikeRate'].iloc[b_idx]
            eco = bowler_info['Economy'].iloc[b_idx]
        else:
            avg = avg_avg
            srate = avg_s_rate
            eco = avg_eco

        b_avgs.append(avg)
        b_srts.append(srate)
        b_eco.append(eco)

        # updating the batter score
        batter = df['Batter'].iloc[idx]
        non_striker = df['Non_Striker'].iloc[idx]

        batter_batting_pos = batting_pos_dict[batter]
        ns_batting_pos = batting_pos_dict[non_striker]

        batter_features = get_batter_features(batter, batter_batting_pos,
                                              (train_batter_avg_dict, train_batter_sr_dict),
                                              (train_grp_avg, train_grp_sr))
        nonstriker_features = get_batter_features(non_striker, ns_batting_pos,
                                                  (train_batter_avg_dict, train_batter_sr_dict),
                                                  (train_grp_avg, train_grp_sr))




        batter_avg.append(batter_features[0])
        batter_sr.append(batter_features[1])
        non_striker_avg.append(nonstriker_features[0])
        non_striker_sr.append(nonstriker_features[1])



    df["Average"] = b_avgs
    df["StrikeRate"] = b_srts
    df["Economy"] = b_eco

    df["Batter_avg"] = batter_avg
    df["Batter_sr"] = batter_sr
    df["NS_avg"] = non_striker_avg
    df["NS_sr"] = non_striker_sr



    # Apply the function to generate dynamic over numbers
    df['Over'] = df.apply(generate_over, axis=1)

    df["Total_Runs"] = df["Batter_Runs"] + df["Extras"]


    df2 = df.loc[:, ['Over', 'Total_Runs', 'Extras', 'Average', 'StrikeRate', 'Economy', 'Batter_avg', 'Batter_sr', 'NS_avg', 'NS_sr']]

    df2['Team_Runs'] = df2['Total_Runs'].cumsum()
    df2['To_Win_Runs'] = target_score - df2['Team_Runs']


    if df2.shape[0]:
        df2 = df2.rename(columns={"Over": 0, "Batter_Runs": 1, "Extras": 2})
        innings2.append(df2)
        if winteam == f_in_team:
            target.append(0)
        else:
            target.append(1)


In [52]:
innings2[0]

,0,Total_Runs,2,Average,StrikeRate,Economy,Batter_avg,Batter_sr,NS_avg,NS_sr,Team_Runs,To_Win_Runs
innings,,,,,,,,,,,,
2,0.1,0,0,19.754098,19.049180,6.243523,33.031250,137.869565,24.897959,118.561710,0,130
2,0.2,1,0,19.754098,19.049180,6.243523,33.031250,137.869565,24.897959,118.561710,1,129
2,0.3,0,0,19.754098,19.049180,6.243523,24.897959,118.561710,33.031250,137.869565,1,129
2,0.4,0,0,19.754098,19.049180,6.243523,24.897959,118.561710,33.031250,137.869565,1,129
2,0.5,0,0,19.754098,19.049180,6.243523,24.897959,118.561710,33.031250,137.869565,1,129
...,...,...,...,...,...,...,...,...,...,...,...,...
2,18.6,1,0,15.888889,13.944444,6.975610,29.666667,136.350975,33.031250,137.869565,129,1
2,19.1,0,0,23.035714,17.535714,7.962963,29.666667,136.350975,33.031250,137.869565,129,1
2,19.2,0,0,23.035714,17.535714,7.962963,29.666667,136.350975,33.031250,137.869565,129,1


In [53]:
# torch tensor processing
features=[]
# create targets first
for i in range(len(innings2)):
    innings2[i] = innings2[i].drop(['Total_Runs', 2, 'Team_Runs'], axis=1)
    x = np.array(innings2[i].values, dtype=np.float16)
    features.append(torch.tensor(x))


# convert to fixed length sequence
features = pad_sequence(features,batch_first=True, padding_value=-1)
targets = torch.tensor(target)

split = int(len(features) * 0.8)
X_train = features[:split]
X_test  = features[split:]
y_train = targets[:split]
y_test  = targets[split:]
print(X_train.size())
print(X_test.size())

torch.Size([2328, 160, 9])
torch.Size([582, 160, 9])


In [54]:
# X_train[0]

In [55]:
y_train.shape, y_test.shape

(torch.Size([2328]), torch.Size([582]))

In [56]:
# LSTM model
class matchRNN(nn.Module):
    def __init__(self,insize,hsize,outsize):
        super(matchRNN,self).__init__()

        self.insize=insize
        self.hsize=hsize
        self.outsize = outsize

        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=insize, hidden_size=hsize)
        self.fc_out = nn.Linear(in_features=hsize, out_features=outsize)
#         self.dropout = nn.Dropout(p=0.2, inplace=False)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,feat):
#         feat = torch.tensor(feat[np.newaxis,:],dtype=torch.float32)
        batch_size = feat.size(0)
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hsize))
        cell_state = torch.zeros((batch_size, self.hsize))
        outputs = torch.empty((batch_size, feat.size(1), self.outsize))

        for t in range(feat.size(1)):

            # for the first time step (if input is different)
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(batch_size,-1).float(), (hidden_state, cell_state))

            # for the 2nd+ time step
            else:
                hidden_state, cell_state = self.lstm_cell(feat[:,t,:].view(batch_size,-1).float(), (hidden_state, cell_state))

#             dropouts = self.dropout(hidden_state)
            out = self.fc_out(hidden_state)
#             out = self.softmax(out)
            outputs[:,t,:] = out

        return outputs

In [57]:
ytrain_tiled = y_train.repeat(X_train.size(1),1).transpose(0,1)
ytrain_tiled.shape

torch.Size([2328, 160])

In [58]:
def evalfunc(model,j):
    model.eval()
    # train
    X = X_train
    y = y_train
    corr=0
    num_batches = X.size(0) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        maxval,maxidx = torch.max(req_op,1)
        corr+= np.sum((maxidx==y[start_idx:end_idx]).numpy())
    total=num_batches*batch_size
    train_acc = corr / total

    # test
    X = X_test
    y = y_test
    corr=0
    num_batches = X.size(0) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        op = model(X[start_idx:end_idx])
        req_op = op[:,j-1]
        maxval,maxidx = torch.max(req_op,1)
        corr+= np.sum((maxidx==y[start_idx:end_idx]).numpy())
    total=num_batches*batch_size
    test_acc = corr / total
    print('j = {}, Train Acc = {}, Test Acc = {}'.format(j,train_acc,test_acc))

In [59]:
from tqdm import tqdm_notebook

In [61]:
### training parameters

insize = X_train.size(2)
hsize = 64
outsize = 2    #binary classification
model = matchRNN(insize,hsize,outsize)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 16
num_batches = int(X_train.size(0) / batch_size)


# train iterations
for epoch in range(121):  # optimum 100-150 epochs
    epoch_loss = 0
    model.train()
    for batch_idx in tqdm_notebook(range(num_batches)):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx+1) * batch_size
        model.zero_grad()
        outputs = model(X_train[start_idx:end_idx])
        loss = loss_function(outputs[:,30:160,:].contiguous().view(-1,outsize), ytrain_tiled[start_idx:end_idx,30:160].contiguous().view(-1))
        loss.backward()
        epoch_loss += loss.data.item()
        optimizer.step()
    print('Epoch = {}, loss = {}'.format(epoch,epoch_loss))
    if epoch % 10 == 0:
        for j in [60, 90, 120, 160]:
            evalfunc(model,j)



  0%|          | 0/145 [00:00<?, ?it/s]

Epoch = 0, loss = 75.59535464644432
j = 60, Train Acc = 0.8176724137931034, Test Acc = 0.8211805555555556
j = 90, Train Acc = 0.8301724137931035, Test Acc = 0.8350694444444444
j = 120, Train Acc = 0.7767241379310345, Test Acc = 0.7777777777777778
j = 160, Train Acc = 0.8370689655172414, Test Acc = 0.8385416666666666


  0%|          | 0/145 [00:00<?, ?it/s]

Epoch = 1, loss = 56.51749262213707


  0%|          | 0/145 [00:00<?, ?it/s]

...

Epoch = 118, loss = 29.719459172338247


  0%|          | 0/145 [00:00<?, ?it/s]

Epoch = 119, loss = 29.472461238503456


  0%|          | 0/145 [00:00<?, ?it/s]

Epoch = 120, loss = 29.646285593509674
j = 60, Train Acc = 0.8504310344827586, Test Acc = 0.8559027777777778
j = 90, Train Acc = 0.9043103448275862, Test Acc = 0.8836805555555556
j = 120, Train Acc = 0.9288793103448276, Test Acc = 0.921875
j = 160, Train Acc = 0.9935344827586207, Test Acc = 0.9861111111111112


In [62]:
torch.save(model.state_dict(), '/content/models/prediction_batter_bowler.pth')